In [18]:
import pandas as pd
from striprtf.striprtf import rtf_to_text
import re
import os
import locale
from datetime import datetime
locale.setlocale(locale.LC_TIME, "nl_NL")
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [20]:
plain_texts = []
folder_path = "data"  ### There needs to be folder in the working directory with the title "data" that has separated rtf.file
file_names = os.listdir(folder_path)
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r") as file:
        text = file.read()
        text = rtf_to_text(text)
        plain_texts.append(text)

In [22]:
dct = {}
for index, t in enumerate(plain_texts): 
    dct[str(index)] = {}
    
    meta = re.search(r"^(.*?)Copyright", t, re.DOTALL)
    section = re.search(r"Section:(.*?)\n", t)
    length = re.search(r"Length:(.*?)\n", t)
    byline = re.search(r"Byline:(.*?)\n", t)
    highlight = re.search(r"Highlight:(.*?)\n", t)
    body = re.search(r"Body(.*?)Load-Date:", t.replace("\n", " "))
    
    if meta:
        meta_data = meta.group(1).strip().split("\n")
        dct[str(index)]["headline"] = meta_data[0]
        dct[str(index)]["news_source"] = meta_data[1]
        dct[str(index)]["datePublished"] = meta_data[2]
        if len(meta_data) > 3:
            dct[str(index)]["region"] = meta_data[3]
    if section:
        dct[str(index)]["articleSection"] = section.group(1).strip()
    if length:
        dct[str(index)]["length"] = length.group(1).strip()
    if byline:
        dct[str(index)]["byline"] = byline.group(1).strip()
    if highlight:
        dct[str(index)]["highlight"] = highlight.group(1).strip()
    if body:
        dct[str(index)]["text"] = body.group(1).strip()

In [24]:
df_print = pd.DataFrame(dct).transpose()

In [38]:
df_print.to_csv("test.csv", encoding='utf-8', index=False)

